In [3]:
import os
from pathlib import Path
import numpy as np
from itertools import product, combinations
from collections import defaultdict
from scipy.spatial import distance_matrix
from scipy.spatial.transform import Rotation #align_vectors

In [5]:
FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day19.txt'

with open(FOLDER / in_file) as f:
    raw = f.read().split('\n\n')
    scanners = []
    for sc in raw:
        l = [line.split(',') for line in sc.split('\n')[1:]]
        scanners.append(np.array(l, dtype=int))


In [6]:
# Distance matrix for each scanner
distances = [np.around(distance_matrix(s, s), decimals=9) for s in scanners]

# So in matrix 0, the distance between the 4th beacon and 7th is:
distances[0][3][6]

2231.102418088

In [27]:
# The scanners we are interested in see 66 pairs of points
# with the same distance (not counting 0 distance with self. 
# 66 is 12 choose 2, so this corresponds to 12 shared points

# for example, scanners 0 and 2 see 12 points with the same distances between them
a_b_shared_distances = np.intersect1d(distances[0], distances[2])[1:]
a_b_shared_distances.shape

(66,)

In [8]:
# Get the pairs we need to examine:

pairs_12 = defaultdict(list)

for i_1, i_2 in combinations(range(len(distances)), r=2):
    a_b_shared_distances = np.intersect1d(distances[i_1], distances[i_2])[1:]
    if a_b_shared_distances.shape[0] == 66:
        pairs_12[i_1].append(i_2)
        pairs_12[i_2].append(i_1)


In [24]:
def get_transform(points_1, points_2):
    '''
    Normalize to the mean first. The take the easy course with scipy!
    This will give us the rotation function and the translation matrix
    '''

    rot, rmsd = Rotation.align_vectors(
        points_1 - np.mean(points_1, axis=0), 
        points_2 - np.mean(points_2, axis=0)
    )
    points_2 = rot.apply(points_2)

    return (points_1 - points_2)[0], rot
    
def find_scanner_locations(points1, points2):
    distances = [np.around(distance_matrix(s, s), decimals=9) for s in [points1, points2]]

    shared_distances = np.intersect1d(distances[0], distances[1])[1:] # remove zero for self distance

    #indices from where these points are in the distance matrix
    ind_1 = np.argwhere(np.isin(distances[0], shared_distances))
    ind_2 = np.argwhere(np.isin(distances[1], shared_distances))

    # the 12 indexes from set of observations
    # but the orders do not necessarily correspond
    ind_1 = np.unique(ind_1.ravel())
    ind_2 = np.unique(ind_2.ravel())

    a_points = points1[ind_1]
    b_points = points2[ind_2]

    a_mean = a_points.mean(axis=0)
    b_mean = b_points.mean(axis=0)

    # sort by distance from local mean
    a_sorted_indices = np.argsort(np.linalg.norm(a_points - a_mean, axis=1))
    b_sorted_indices = np.argsort(np.linalg.norm(b_points - b_mean, axis=1))

    points_1 = a_points[a_sorted_indices]
    points_2 = b_points[b_sorted_indices]

    return get_transform(points_1, points_2)

def transform_points(sensor_number, scanners, scanner_data):
    '''Just apply rotation and translation. No scaling needed'''
    loc, rot = scanner_data[sensor_number]
    return loc + rot.apply(scanners[sensor_number])

def get_unique(pair_dict):
    references = {
        0: scanners[0].copy()
    }
    scanner_data = {
        0: ([0,0,0], Rotation.from_matrix(np.zeros([3, 3])))
    }
    
    keys_seen = set()
    to_find = [0]
    
    while to_find:
        '''do a simple graph search to find the right order to lookup'''
        reference = to_find.pop(0)

        if reference in keys_seen:
            continue

        keys_seen.add(reference)

        for sensor in pair_dict[reference]:
            if sensor in scanner_data:
                continue
            to_find.append(sensor)
        
            points = references[reference] # these should already be transformed
            loc, rot = find_scanner_locations(references[reference], scanners[sensor])            

            loc = np.rint(loc).astype(np.int32) # careful to rount before casting

            scanner_data[sensor] = (loc, rot)
            translated = transform_points(sensor,scanners,scanner_data)

            references[sensor] = translated
        
    return references, scanner_data

references, scanner_data = get_unique(pairs_12)


In [25]:
# Make a set of unique points

known = set(tuple(l) for l in scanners[0])

for k, l in pairs_12.items():
    for sensor in l:
        if sensor == 0: # don't add 0...already started with it
            continue
        translated = transform_points(sensor,scanners,scanner_data)
        for t in translated:
            known.add(tuple(np.rint(t).astype(int)))

print("Solution 1: ", len(known) )

Solution 1:  405


In [26]:
# Minkowski p-norm of 1 corresponds to Manhattan Distance
# We can just make another distance matrix

locations = [s[0] for s in scanner_data.values()]
largest = distance_matrix(locations, locations, p=1).max()

print("Solution 2: ", int(largest))

Solution 2:  12306
